In [1]:
import csv
import math
import random

In [2]:
# Majority Function which tells which class has more entries in given data-set
def majorClass(data):
    # seprates the no of yes instances and no of no instances. return yes if yes is more or no if no
    # is more
    freq = {}
    for tuple in data:
        freq[tuple[-1]] = freq.get(tuple[-1], 0) + 1

    return max(freq, key = freq.get)

In [3]:
# Calculates the entropy of the data given the target attribute
def entropy(attributes, data, target):

    freq = {}
    dataEntropy = 0.0
    i = 0
    for attri in attributes:
        if (target == attri):
            break
        i = i + 1
    i = i - 1
    # classifies data on the basis of each attribute index of target attribute which is 
    # target attribute
    for entry in data:
        freq[entry[i]] = freq.get(entry[i], 0.0) + 1.0
    
    for val in freq.values():
        dataEntropy += (-val/len(data)) * math.log(val/len(data), 2) 
        
    return dataEntropy

In [4]:
# Calculates the information gain (reduction in entropy) in the data when a particular attribute is chosen for splitting the data.
def info_gain(attributes, data, attr, target):

    freq = {}
    subsetEntropy = 0.0
    i = attributes.index(attr)

    for entry in data:
        freq[entry[i]] = freq.get(entry[i], 0.0) + 1.0

    for key in freq.keys():
        valProb        = freq[key] / sum(freq.values())
        dataSubset     = [entry for entry in data if entry[i] == key]
        subsetEntropy += valProb * entropy(attributes, dataSubset, target)

    return (entropy(attributes, data, target) - subsetEntropy)

In [5]:
# This function chooses the attribute among the remaining attributes which has the maximum information gain.
def attr_choose(data, attributes, target):

    best = attributes[0]
    maxGain = 0

    for attr in attributes:
        newGain = info_gain(attributes, data, attr, target) 
        if newGain>maxGain:
            maxGain = newGain
            best = attr

    return best


In [6]:
# This function will get unique values for that particular attribute from the given data
def get_values(data, attributes, best):

    index = attributes.index(best)
    values = []

    for entry in data:
        if entry[index] not in values:
            values.append(entry[index])

    return values

In [7]:
# This function will get all the rows of the data where the chosen "best" attribute has a value "val"
def get_data(data, attributes, best, val):

    new_data = [[]]
    index = attributes.index(best)

    for entry in data:
        if (entry[index] == val):
            newEntry = []
            for i in range(0,len(entry)):
                if(i != index):
                    newEntry.append(entry[i])
            new_data.append(newEntry)

    new_data.remove([])    
    return new_data

In [8]:
# This function is used to build the decision tree using the given data, attributes and the target attributes. It returns the decision tree in the end.
def build_tree(data, attributes, target):

    vals = [record[-1] for record in data]
    default = majorClass(data)

    # if there is only yes or no in dataset return yes or no.
    if vals.count(vals[0]) == len(vals):
        return vals[0]
    else:
        best = attr_choose(data, attributes, target)
        tree = {best:{}}
    
        for val in get_values(data, attributes, best):
            new_data = get_data(data, attributes, best, val)
            newAttr = attributes[:]
            newAttr.remove(best)
            subtree = build_tree(new_data, newAttr, target)
            tree[best][val] = subtree
    
    return tree

In [9]:
#Main function
def execute_decision_tree():
	
    data = []

    #load file
    with open("weather1.csv") as tsv:
        for line in csv.reader(tsv):    
            data.append(tuple(line))
        print("Number of records:",len(data))

        #set attributes
        attributes=['outlook','temperature','humidity','wind','play']
        target = attributes[-1]

        #set training data
        acc = []
        training_set = data.copy()
        tree = build_tree( training_set, attributes, target )
        print(tree)

        #execute algorithm on test data
        results = []
        test_set = [('overcast','cool','normal','strong')]
        
        for entry in test_set:
            tempDict = tree.copy()
            result = ""
            while(isinstance(tempDict, dict)):
                child=[]
                nodeVal=next(iter(tempDict))
                child=tempDict[next(iter(tempDict))].keys()
                tempDict = tempDict[next(iter(tempDict))]
                index = attributes.index(nodeVal)
                value = entry[index]
                if(value in tempDict.keys()):
                    result = tempDict[value]
                    tempDict = tempDict[value]
                else:
                    result = "Null"
                    break
            if result != "Null":
                results.append(result == entry[-1])
        print(result)

In [10]:
if __name__ == "__main__":
	execute_decision_tree()

Number of records: 14
{'wind': {'wind': 'play', 'weak': {'humidity': {'high': {'outlook': {'overcast': 'yes', 'rainy': 'yes', 'sunny': 'no'}}, 'normal': 'yes'}}, 'strong': {'humidity': {'high': {'outlook': {'sunny': 'no', 'overcast': 'yes', 'rainy': 'no'}}, 'normal': {'outlook': {'rainy': 'no', 'overcast': 'yes', 'sunny': 'yes'}}}}}}
yes


In [11]:
def info_gain(data, attribute, attri, target):
    freq = {}
    subEnt = 0.0
    i = attribute.index(attri)
    for entry in data:
        freq[entry[i]] = freq.get(entry[i], 0.0) + 1.0
        
    for key in freq.keys():
        prob = freq[key]/sum(freq.values())
        datasub = [entry for entry in data if entry[i] == key]
        subEnt = prob*entropy(datasub, attribute, target)
    
    return (entropy(data, attribute, target) - subEnt)